Kickstart the simulation.

In [ ]:
#import logging as _logging_
#_logging_.basicConfig(level='INFO')
#from energyplus.ooep.addons.logging import LogProvider

from energyplus.ooep.addons.progress import ProgressProvider

from energyplus.ooep import (
    Simulator,
    Model,
    Weather,
    Report,
)

from energyplus.dataset.basic import dataset as _epds_

simulator = Simulator().add(
    ProgressProvider(),
    #LogProvider(),
)

simulator.awaitable.run_forever(
    input=Simulator.InputSpecs(
        model=(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=(_epds_.weathers / 'USA_CO_Denver-Aurora-Buckley.AFB.724695_TMY3.epw'),
    ),
    output=Simulator.OutputSpecs(
        #report=('/tmp/ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    options=Simulator.RuntimeOptions(
        #design_day=True,
    ),
)

method 1

In [ ]:
import energyplus.ooep as _ooep_

obs = simulator.variables.getdefault(
    _ooep_.OutputVariable.Ref(
        type='People Air Temperature',
        key='CORE_MID',
    )
).value

simulator.variables.getdefault(
    _ooep_.Actuator.Ref(
        type='Zone Temperature Control',
        control_type='Heating Setpoint',
        key='CORE_MID',
    )
).value = 16
actuator = simulator.variables.getdefault(
    _ooep_.Actuator.Ref(
        type='Zone Temperature Control',
        control_type='Heating Setpoint',
        key='CORE_MID',
    )
).value 
print(obs,actuator)


method 2

In [ ]:
import numpy as _numpy_
import gymnasium as _gymnasium_

from energyplus.ooep.addons.rl import (
    VariableBox,
    SimulatorEnv,
)

from energyplus.ooep import (
    Actuator,
    OutputVariable,
)
from energyplus.ooep.addons.rl.gymnasium import ThinEnv

simulator.add(
    thinenv := ThinEnv(
        action_space=_gymnasium_.spaces.Dict({
                    'thermostat': VariableBox(
                        low=15., high=16.,
                        dtype=_numpy_.float32,
                        shape=(),
                    ).bind(Actuator.Ref(
                        type='Zone Temperature Control',
                        control_type='Heating Setpoint',
                        key='CORE_MID',
                    ))
                }),    
        observation_space=_gymnasium_.spaces.Dict({
            'temperature': VariableBox(
                low=-_numpy_.inf, high=+_numpy_.inf,
                dtype=_numpy_.float32,
                shape=(),
            ).bind(OutputVariable.Ref(
                type='People Air Temperature',
                key='CORE_MID',
            )),
        }),
    )
)
obs = thinenv.observe()
act = thinenv.act({'thermostat': 16. })
print(obs,act)